In [1]:
import copy
import numpy as np
import pandas as pd
import scanpy as sc
from MarkerCount.marker_count import MarkerCount_Ref, MarkerCount

https://github.com/combio-dku


In [2]:
from importlib.metadata import version

In [3]:
version('pandas')

#require use of 1.4.3 panda

'1.4.3'

#### Import markers

In [4]:
#file_marker_matrix = './../data/gbm_subtye_genelist_markercount.xlsx'
file_marker_matrix = './../data/gbm_MESImm_markercount.xlsx'
#file_marker_matrix = './../data/gbm_subtype_new_markercount.xlsx'
#file_marker_matrix = './../data/gbm_subtype_new_50_markercount.xlsx'

df_mkr_mat = pd.read_excel(file_marker_matrix, index_col = 0, sheet_name=0)

#This is for subsheet which have 4 subtype (MES1 + MES2 = MES, and also combined NPC)
#df_mkr_mat = pd.read_excel(file_marker_matrix, index_col = 0, sheet_name=1)

df_mkr_mat.fillna(0,inplace=True)
df_mkr_mat= df_mkr_mat.astype(int)

In [5]:
df_mkr_mat

,TGFBI,COL14A1,MYL9,TNFAIP2,IGFBP6,COL3A1,SULF1,ADAMTSL1,VAT1L,OAS2,...,MARCKSL1,ARHGEF9,CASK,ST8SIA1,ZNF776,LANCL2,SPHK2,ZNF45,TCF12,ARHGEF7
celltype,,,,,,,,,,,,,,,,,,,,,
MESImm,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Non_MESImm,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


#### Import dataset

In [6]:
file_test = './../output/gbm_qc.h5ad'
adata_test = sc.read_h5ad(file_test)

In [7]:
adata_test.obs['split']

run1_control_AAACCCAAGACATAGT-1      run1_control_E55N
run1_control_AAACCCAAGCGCCATC-1      run1_control_E57N
run1_control_AAACCCACAAGCTGTT-1      run1_control_E55N
run1_control_AAACCCACACCACTGG-1      run1_control_E57N
run1_control_AAACCCACATGCGTGC-1      run1_control_E57N
                                           ...        
run2_radiated_TTTGTTGGTGAGATAT-1    run2_radiated_E35N
run2_radiated_TTTGTTGGTGTTGACT-1    run2_radiated_E31N
run2_radiated_TTTGTTGGTTCAAACC-1    run2_radiated_E31N
run2_radiated_TTTGTTGTCACTACGA-1    run2_radiated_E37N
run2_radiated_TTTGTTGTCGAACTCA-1    run2_radiated_E56N
Name: split, Length: 20043, dtype: object

#### Now this time calculate each seperately (but automatatically)

In [8]:
groups = adata_test.obs.groupby("split").indices

groups["run2_radiated_E31N"] = np.concatenate((groups["run2_radiated_E31N"], groups["run1_radiated_E31N"]))
groups["run2_control_E31N"] = np.concatenate((groups["run2_control_E31N"], groups["run1_control_E31N"]))
groups["run2_radiated_E26N"] = np.concatenate((groups["run2_radiated_E26N"], groups["run1_radiated_E26N"]))
groups["run2_radiated_E24N"] = np.concatenate((groups["run2_radiated_E24N"], groups["run1_radiated_E24N"]))


keys = ["run1_radiated_E24N","run1_radiated_E26N","run1_control_E31N","run1_radiated_E31N"]

for key in keys:
    groups.pop(key, None)

In [9]:
res_list= []
pred_list = []
for i in groups:
    each = adata_test[groups[i]]
    print("running: "+i)
    #Just same code
    X_test = each.to_df() # rows: cell, cols: genes
    df_res = MarkerCount(X_test, df_mkr_mat, \
                          log_transformed = False, verbose = False )  

    res_list.append(df_res)

    each.obs['markercount'] = df_res['cell_type_pred'] 
    export = each.obs[['markercount']]

    pred_list.append(export)
    print("done: "+i)

running: run1_control_E13N


/home/point/anaconda3/lib/python3.10/site-packages/MarkerCount/marker_count.py:1045: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  mkrs = get_best_n_markers( df_ofreq_tmp_all.loc[:,ct_mkrs], ct, of_th = of_th, \
/tmp/ipykernel_206440/2094353085.py:13: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  each.obs['markercount'] = df_res['cell_type_pred']


done: run1_control_E13N
running: run1_control_E51N


/home/point/anaconda3/lib/python3.10/site-packages/MarkerCount/marker_count.py:1045: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  mkrs = get_best_n_markers( df_ofreq_tmp_all.loc[:,ct_mkrs], ct, of_th = of_th, \
/tmp/ipykernel_206440/2094353085.py:13: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  each.obs['markercount'] = df_res['cell_type_pred']


done: run1_control_E51N
running: run1_control_E53N


...

/home/point/anaconda3/lib/python3.10/site-packages/MarkerCount/marker_count.py:1045: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  mkrs = get_best_n_markers( df_ofreq_tmp_all.loc[:,ct_mkrs], ct, of_th = of_th, \
/home/point/anaconda3/lib/python3.10/site-packages/MarkerCount/marker_count.py:1045: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  mkrs = get_best_n_markers( df_ofreq_tmp_all.loc[:,ct_mkrs], ct, of_th = of_th, \
/tmp/ipykernel_206440/2094353085.py:13: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  each.obs['markercount'] = df_res['cell_type_pred']


done: run2_radiated_E39N
running: run2_radiated_E56N


/home/point/anaconda3/lib/python3.10/site-packages/MarkerCount/marker_count.py:1045: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  mkrs = get_best_n_markers( df_ofreq_tmp_all.loc[:,ct_mkrs], ct, of_th = of_th, \
/tmp/ipykernel_206440/2094353085.py:13: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  each.obs['markercount'] = df_res['cell_type_pred']


done: run2_radiated_E56N
running: run2_radiated_E62N
done: run2_radiated_E62N


/home/point/anaconda3/lib/python3.10/site-packages/MarkerCount/marker_count.py:1045: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  mkrs = get_best_n_markers( df_ofreq_tmp_all.loc[:,ct_mkrs], ct, of_th = of_th, \
/home/point/anaconda3/lib/python3.10/site-packages/MarkerCount/marker_count.py:1045: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  mkrs = get_best_n_markers( df_ofreq_tmp_all.loc[:,ct_mkrs], ct, of_th = of_th, \
/tmp/ipykernel_206440/2094353085.py:13: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  each.obs['markercount'] = df_res['cell_type_pred']


In [10]:
pred_all = pd.concat(pred_list)

In [11]:
pred_all

,markercount
run1_control_AAACGAAGTGGATTTC-1,MESImm
run1_control_AAACGCTGTATCGAAA-1,MESImm
run1_control_AAACGCTTCTGGCTGG-1,MESImm
run1_control_AAAGAACAGATCGGTG-1,MESImm
run1_control_AAAGGGCAGGTCCGAA-1,MESImm
...,...
run2_radiated_TTGAGTGCAGGTGTGA-1,Non_MESImm
run2_radiated_TTGGTTTCATCGAGCC-1,Non_MESImm
run2_radiated_TTTACCAAGCAGTACG-1,Non_MESImm
run2_radiated_TTTGGTTAGACCACGA-1,unassigned


In [12]:
# Check whether there is a duplication of cell_id 
cell_id = pred_all.index.values

u, c = np.unique(cell_id, return_counts=True)
dup = u[c > 1]

len(dup)

0

In [13]:
# Yeah, there are
# so it is reasonable to convert rowname to first column as "R" dataframe cannot handle replicated rownames

pred_all.index.name = 'cell_id'
pred_all.reset_index(inplace=True)

In [14]:
pred_all.head()

,cell_id,markercount
0,run1_control_AAACGAAGTGGATTTC-1,MESImm
1,run1_control_AAACGCTGTATCGAAA-1,MESImm
2,run1_control_AAACGCTTCTGGCTGG-1,MESImm
3,run1_control_AAAGAACAGATCGGTG-1,MESImm
4,run1_control_AAAGGGCAGGTCCGAA-1,MESImm


In [15]:
# pred_all.to_csv('./../output/markercount.csv')
#pred_all.to_csv('./../output/markercount_all_6metamodules.csv',index=False)
pred_all.to_csv('./../output/markercount_all_MESImm.csv',index=False)
#pred_all.to_csv('./../output/markercount_all_4metamodules_new.csv',index=False)
#pred_all.to_csv('./../output/markercount_all_4metamodules_new_50.csv',index=False)